### Visualizando a Transformada de Fourier (EPO application)
(Baseado em: [But what is the Fourier Transform? A visual introduction.](https://youtu.be/spUNpyF58BY)) <br>
rcboufleur at gmail com | 2020


**Transformada de Fourier** (equação de análise):


$$ X(\omega)=\int_{-\infty}^{+\infty} x(t) e^{-j \omega t} d t $$

onde, a partir da fórmula de Euler, $e^{-j \omega t} = \cos{\omega t} -j\sin{\omega t}$.

In [2]:
import numpy as np
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import figure, output_notebook, show

output_notebook()

# definir parametros do sinal
ruido = False
subtrair_media = True

amp = [1.0, 2.0, 3.6]
f = [2.0, 5.0, 3.5]
phase = [0.1, 0.27, 0.7]
h = 10.0    # valor médio adicionado ao sinal

sampling = 0.01    # amostragem do sinal em seg/ciclo

fini = 0.0   # frequência inicial a ser buscada
fend = 120    # frequência final a ser buscada
fstep = 0.01 # passo em frequência

nyqst = 1/(2*sampling)
print('Limite de Nyqst: {0:5.1f} ciclos/segundo'.format(nyqst))
# construir variáveis
nf = len(f)
x = np.arange(0,5,sampling)  # variavel independente x
ys = np.zeros((nf,len(x)))   # array para conter sinais individuais
y = np.zeros(len(x))         # array contendo sinais somados
for i in range(nf):
    ys[i,:] = amp[i]*np.sin(2*np.pi*(f[i]*x + phase[i]))
    y += ys[i,:]
y += h                       # adiciona valor médio do sinal


# definir funcoes usadas dentro do js (devem ser declaradas antes do callback)
im, re = np.zeros(len(x)),np.zeros(len(x))
mgi =  np.linspace(0,1,len(x))
mgr =  np.linspace(0,1,len(x))
magf = np.linspace(0,1,len(x))
fmag = np.zeros(len(x))
yfit = np.zeros(len(x))

freq = np.arange(fini,fend,fstep)
mag = np.zeros(len(freq))
theta = np.zeros(len(freq))

noise = np.random.normal(0,2,len(x))  # ruido gaussiano com média 0 e desvio padrao 2
if ruido:
    y += noise

if subtrair_media:
    y -= np.mean(y)

    

    
# calculo do objeto interativo
source = ColumnDataSource(data=dict(x=x, y=y, im=im, re=re, mgi=mgi, \
                                    mgr=mgr, magf=magf, fmag=fmag, yfit=yfit))
# plota sinais separados
s0 = figure(plot_width=400, plot_height=165, x_range=(x.min(),x.max()), title='Sinal Arbitrário')

ran = np.zeros(len(x))
for i in range(nf):
    ran += ys[i,:].max()-ys[i,:].min()
    s0.line(x, ys[i,:]+ran+3)
s0.axis.visible=False
# plota sinais somados
s3 = figure(plot_width=400, plot_height=165, x_range=(x.min(),x.max()))
s3.line(x, y)
s3.line('x', 'yfit', source=source, line_alpha=0.8, color='red')
s3.yaxis.axis_label = "Intensidade"
s3.xaxis.axis_label = "Tempo (segundos)"


rg = 1.25*y.max()

# calcula dft
for j,i in enumerate(freq):
    r1 = y*np.cos(-2*np.pi*x*i)/len(x)
    im1 = y*np.sin(-2*np.pi*x*i)/len(x)  
    r1 = np.sum(r1)
    im1 = np.sum(im1)
    mag[j] = 2*np.sqrt(r1**2 + im1**2)

# plota plano
s1 = figure(plot_width=300, plot_height=270,x_range=(-rg,rg), y_range=(-rg,rg))
s1.line(x=[0,0], y=[-rg,rg], color='black')
s1.line(x=[-rg,rg], y=[0,0], color='black')
s1.line('re', 'im', source=source, color='blue', line_alpha=0.2)
s1.circle('re', 'im', source=source, color='blue', alpha=0.2)
s1.circle('mgr','mgi', source=source, color='red', size=5)
s1.xaxis.axis_label = "Real"
s1.yaxis.axis_label = "Imaginário"


nf = np.array([nyqst,nyqst])
np = np.array([0,1.2*mag.max()])
s2 = figure(plot_width=700, plot_height=250,x_range=(fini,fend), y_range=(0,1.05*mag.max()))
s2.line(freq, mag, color='green', line_width=1.5)
s2.circle('fmag','magf', source=source, color='red', size=5)
s2.line(x=nf, y=np, color='blue', legend_label='Limite de Nyqst')
s2.legend.location = "top_right"
s2.yaxis.axis_label = "Amplitude"
s2.xaxis.axis_label = "Frequência (ciclos/segundo)"


callback = CustomJS(args=dict(source=source), code="""
        const arrSum = arr => arr.reduce((a,b) => a + b, 0);
        var data = source.data;
        var f = cb_obj.value;
        var x = data['x'];
        var y = data['y'];
        var im = data['im'];
        var re = data['re'];
        var mgi = data['mgi'];
        var mgr = data['mgr'];
        var magf = data['magf']
        var fmag = data['fmag']
        var yfit = data['yfit']
        for (var i = 0; i < x.length; i++){
            re[i] = y[i]*Math.cos(-2*Math.PI*x[i]*f)
            im[i] = y[i]*Math.sin(-2*Math.PI*x[i]*f)
        }
        let ret = arrSum(re)/x.length;
        let imt = arrSum(im)/x.length;
        let mg = Math.sqrt(ret*ret + imt*imt);

        var tfix = 0;
        if(imt>0 && ret<0){
            tfix = Math.PI;
        } else if(imt<0 && ret<0){
            tfix = 3*Math.PI/2;
        } else if(imt<0 && ret>0){
            tfix = 2*Math.PI;
        }
        let theta = Math.atan(imt/ret) + tfix


        for (var i = 0; i < x.length; i++){
            mgi[i] = imt;
            mgr[i] = ret;
        //    magf[i] = mg*2*(i/x.length)
            magf[i] = mg*2
            fmag[i] = f;
        }

        for (var i = 0; i < x.length; i++){
            yfit[i] = 2*mg*Math.cos(-2*Math.PI*((f*x[i]) + theta/(2*Math.PI)))
        }
        source.change.emit();
    """)

slider = Slider(start=fini, end=fend, value=0, step=fstep, title="Ciclos por segundo")
slider.js_on_change('value', callback)

layout = column(row(column(s0,s3), column(slider, s1)), s2, width=900)
show(layout)

Loading BokehJS ...

Limite de Nyqst:  50.0 ciclos/segundo
